<a href="https://colab.research.google.com/github/vardhannegi/Task1/blob/master/Quantum_circuits_task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install qiskit

     |████████████████████████████████| 6.4 MB 25.7 MB/s 
     |████████████████████████████████| 18.0 MB 464 kB/s 
     |████████████████████████████████| 238 kB 66.2 MB/s 
     |████████████████████████████████| 200 kB 65.1 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 1.4 MB 45.7 MB/s 
     |████████████████████████████████| 38.2 MB 1.2 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
     |████████████████████████████████| 49 kB 5.6 MB/s 
     |████████████████████████████████| 49 kB 5.4 MB/s 
     |████████████████████████████████| 943 kB 48.0 MB/s 
     |████████████████████████████████| 3.6 MB 47.5 MB/s 
     |████████████████████████████████| 112 kB 72.6 MB/s 
  Created wheel for qiskit: filename=qiskit-0.34.0-py3-none-any.whl size=11764 sha256=a929d51156599262c8aa1d802e8e28673c00119615c7bf96a09c6bc4e33051d0
  Stored in directory: /root/.cache/pip/wheels/be/f4/26/53ffcc0c3a1303116016eaf20c4f0c8dc125bc3b3c7af

In [3]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, execute
from numpy import linalg as LA
import pandas as pd
import numpy as np
import pprint
import random
import time


In [4]:
ep = []
layer_count = 0
final_theta ={}
circuitlist = []
simulator = Aer.get_backend('statevector_simulator')
y_upper = int((8+2)/2)
theta_range = np.linspace(0, 2 * np.pi, 128)
random_state = np.array([1,0,1,1,1,0,0,0,0,0,1,1,0,1,0,0])

class circuitgen:
    def __init__(self,total_layer,total_instance):
        self.n = total_layer 
        self.m = total_instance   
            
        for x in range(1,self.m+1):
            
            theta_dict = {f'instance{x}': {}}
            for y in range(1,8*self.n+1):
              theta_dict[f'instance{x}']['{0}'.format(str(y).zfill(2))] = random.choice(theta_range)
              final_theta.update(theta_dict)
              
            cap = 0
            qc = f'qc{x}'
            qc = QuantumCircuit(4,4)
            
            for _ in range(1,self.n+1):
              for y in range(1,y_upper):
                zz = cap + y
                a = '{0}'.format(str(zz).zfill(2))
                # print(a)
                qc.rx(final_theta[f'instance{x}'][a],y-1)

              for num in range(4):
                if num<=2:
                  qc.cz(0,num+1)

              for num in range(4):
                if num<2:
                  qc.cz(1,(num+1)+1)
              qc.cz(2,3)
              for y in range(y_upper,8+1):
                if y <8:
                  zz = cap + y
                  a = '{0}'.format(str(zz).zfill(2))
                  # print(a)
                  qc.rx(final_theta[f'instance{x}'][a],y-5)
                else :
                  zz = cap + y
                  a = '{0}'.format(str(zz).zfill(2))
                  # print(a)
                  qc.rx(final_theta[f'instance{x}'][a],y-5)
                  cap = zz
              qc.barrier()
            
            circuitlist.append(qc)
##           return circuitlist
          
    def getInputData(self):
        for x in circuitlist:
              result = execute(x, simulator).result()
              statevector = result.get_statevector(x)
              epsilon = LA.norm(statevector - random_state)
              ep.append(epsilon)
        df = pd.DataFrame.from_dict(final_theta)
        kk = df.T
        kk['epsilon'] = ep 
        return kk
      

##kk.to_csv('2x10kmodel.csv')



In [13]:
circuitgen(3,4)
n = len(circuitlist)
for i in range(0,n):
  print(circuitlist[i])

     ┌────────────┐         ┌────────────┐                                ░ »
q_0: ┤ Rx(3.6611) ├─■──■──■─┤ Rx(2.6221) ├────────────────────────────────░─»
     └─┬────────┬─┘ │  │  │ └────────────┘   ┌────────────┐               ░ »
q_1: ──┤ Rx(2π) ├───■──┼──┼───────■────────■─┤ Rx(2.5232) ├───────────────░─»
      ┌┴────────┴┐     │  │       │        │ └────────────┘┌────────────┐ ░ »
q_2: ─┤ Rx(2.82) ├─────■──┼───────■────────┼───────■───────┤ Rx(2.9684) ├─░─»
     ┌┴──────────┴┐       │                │       │       └┬──────────┬┘ ░ »
q_3: ┤ Rx(3.6611) ├───────■────────────────■───────■────────┤ Rx(1.88) ├──░─»
     └────────────┘                                         └──────────┘  ░ »
c: 4/═══════════════════════════════════════════════════════════════════════»
                                                                            »
«      ┌────────────┐         ┌────────────┐                                 ░ »
«q_0: ─┤ Rx(4.6011) ├─■──■──■─┤ Rx(3.1169) ├─────────────────